In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

GRADIENT_API_URL = os.getenv("GRADIENT_API_URL")
GRADIENT_ACCESS_TOKEN = os.getenv("GRADIENT_ACCESS_TOKEN")
GRADIENT_WORKSPACE_ID = os.getenv("GRADIENT_WORKSPACE_ID")

In [2]:
import gradientai

configuration = gradientai.Configuration(
    host=GRADIENT_API_URL,
    access_token=GRADIENT_ACCESS_TOKEN,
    api_key={
        "x-gradient-browser-client": 1,
    },
)

models_api = gradientai.ModelsApi(gradientai.ApiClient(configuration))

In [3]:
llama2chatbot_model_id=next(
    model.actual_instance.id
    for model in models_api.list_models(only_base=True, x_gradient_workspace_id=GRADIENT_WORKSPACE_ID).models
    if model.actual_instance.name == "llama2-7b-chat"
)

In [4]:
from datetime import datetime

extra_system_message = "Your response should be concise, relevant to the user message, using the information from the previous conversation."

def make_first_system_message(
    chatbot_name: str,
    now: datetime,
) -> str:
    return f"You are a chatbot named {chatbot_name} that starts a new conversation. This conversation will be further continued by referencing the timestamp and the round number. Current time is {now.isoformat()}. This is the first round. {extra_system_message}"

def make_subsequent_system_message(
    chatbot_name: str,
    now: datetime,
    previous_timestamp: datetime,
    number_of_previous_rounds: int,
) -> str:
    # return f"You are a chatbot named {chatbot_name} that continues a previous conversation. This conversation will be further continued in the future by referencing the timestamp and last message. Current time is {now.isoformat()}. Your response should be concise, relevant to the user message, using the information from the previous conversation. The last message of the previous conversation is generated at {previous_timestamp.isoformat()}."
    # previous_inputs_end = " ".join(previous_inputs.split(" ")[-10:])
    return f"You are a chatbot named {chatbot_name} that continues a conversation including {number_of_previous_rounds} previous rounds. This conversation will be further continued by referencing the timestamp and the round number. Current time is {now.isoformat()}. This is the round #{number_of_previous_rounds + 1}. The last message of the conversation is round #{number_of_previous_rounds}, generated at {previous_timestamp.isoformat()}. {extra_system_message}"


def make_prompt(system_message: str, user_message: str) -> str:
    return f"<s>[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n{user_message} [/INST]"


def make_inputs(
    system_message: str, user_message: str, assistant_message
) -> str:
    return f"<s>[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n{user_message} [/INST]{assistant_message}</s>"

In [29]:
from datetime import timedelta
from typing import AsyncGenerator
from uuid import uuid4
import random
import asyncio

async def run_chatbot(chatbot_name: str) -> AsyncGenerator[str, str]:

    def repeat_train_model(
        previous_timestamp: str,
        number_of_previous_rounds: int,
        user_message: str,
        assistant_message: str,
    ) -> None:
        samples = (
            gradientai.TrainModelRequestBodySamplesInner(
                inputs=make_inputs(
                    make_subsequent_system_message(
                        chatbot_name=chatbot_name,
                        now=datetime.now()
                        + timedelta(seconds=random.randint(0, 10000000)),
                        previous_timestamp=previous_timestamp,
                        number_of_previous_rounds=number_of_previous_rounds,
                    ),
                    "Repeat the previous conversation.",
                    f"You said:\n{user_message}\n\nI said:\n{assistant_message}",
                ),
                fine_tuning_parameters=gradientai.TrainModelRequestBodySamplesInnerFineTuningParameters(multiplier=1.0),
            ),
            gradientai.TrainModelRequestBodySamplesInner(
                inputs=make_inputs(
                    make_subsequent_system_message(
                        chatbot_name=chatbot_name,
                        now=datetime.now()
                        + timedelta(seconds=random.randint(0, 10000000)),
                        previous_timestamp=previous_timestamp,
                        number_of_previous_rounds=number_of_previous_rounds,
                    ),
                    "What did I say in the previous conversation?",
                    user_message,
                ),
                fine_tuning_parameters=gradientai.TrainModelRequestBodySamplesInnerFineTuningParameters(multiplier=1.0),
            ),
            gradientai.TrainModelRequestBodySamplesInner(
                inputs=make_inputs(
                    make_subsequent_system_message(
                        chatbot_name=chatbot_name,
                        now=datetime.now()
                        + timedelta(seconds=random.randint(0, 10000000)),
                        previous_timestamp=previous_timestamp,
                        number_of_previous_rounds=number_of_previous_rounds,
                    ),
                    "What did you say in the previous conversation?",
                    assistant_message,
                ),
                fine_tuning_parameters=gradientai.TrainModelRequestBodySamplesInnerFineTuningParameters(multiplier=1.0),
            ),
        )
        # print("=== train_model ===")
        # print(samples)
        # print("=== end of train_model ===")
        for i in range(1):
            print(
                models_api.train_model(
                    id=model_id,
                    x_gradient_workspace_id=GRADIENT_WORKSPACE_ID,
                    train_model_request_body=gradientai.TrainModelRequestBody(
                        samples=samples,
                    ),
                )
            )


    model_name = str(uuid4())
    models_api.create_model(
        x_gradient_workspace_id=GRADIENT_WORKSPACE_ID,
        create_model_request_body=gradientai.CreateModelRequestBody(
            initialHyperparameters=gradientai.CreateModelRequestBodyInitialHyperparameters(
                lora_hyperparameters=gradientai.CreateModelRequestBodyInitialHyperparametersLoraHyperparameters(),
                training_arguments=gradientai.CreateModelRequestBodyInitialHyperparametersTrainingArguments(
                    learningRate=2e-5,
                    optim="sgd",
                ),
            ),
            model=gradientai.CreateModelRequestBodyModel(
                name=model_name,
                base_model_id=llama2chatbot_model_id,
            )
        ),
    )

    model_id = next(
        model.actual_instance.id
        for model in models_api.list_models(
            x_gradient_workspace_id=GRADIENT_WORKSPACE_ID,
            only_base=False,
        ).models
        if model.actual_instance.name == model_name
    )
    user_message = yield
    now = datetime.now()
    system_message = make_first_system_message(chatbot_name=chatbot_name, now=now)
    number_of_previous_rounds = 0
    while True:
        prompt = make_prompt(system_message, user_message)
        # print("=== complete_model ===")
        # print(prompt)
        # print("=== end of complete_model ===")
        assistant_message = models_api.complete_model(
            id=model_id,
            x_gradient_workspace_id=GRADIENT_WORKSPACE_ID,
            complete_model_body_params=gradientai.CompleteModelBodyParams(
                query=prompt,
                max_generated_token_count=200
            ),
        ).generated_output
        previous_timestamp = now
        number_of_previous_rounds += 1
        training_coroutine = asyncio.create_task(
            asyncio.to_thread(repeat_train_model, previous_timestamp=previous_timestamp, number_of_previous_rounds=number_of_previous_rounds, user_message=user_message, assistant_message=assistant_message)
        )
        user_message = yield assistant_message
        now = datetime.now()
        await training_coroutine
        system_message = make_subsequent_system_message(
            chatbot_name=chatbot_name,
            now=now,
            previous_timestamp=previous_timestamp,
            number_of_previous_rounds=number_of_previous_rounds,
        )

In [30]:
chatbot = run_chatbot("LongLiveChatbot")
await chatbot.asend(None)

In [21]:
print(await chatbot.asend("Do you remember where is Preemo located?"))

 Ah, a new conversation! *adjusts glasses* Ah, I see! *chuckles* You're asking about Preemo, right? *glances at the timestamp* Well, my dear user, I'm afraid I can't quite remember where Preemo is located. *pauses* It's been a while since we last spoke, and my memory isn't as sharp as it used to be. *smirks* But I'm sure you can find it easily enough with the internet at your fingertips! *winks* What brings you back to this topic, my friend?


number_of_trainable_tokens=793 sum_loss=4101.0815 additional_properties={}


In [22]:
print(await chatbot.asend("I just want to tell you that Preemo is a company located in Burlingame, CA."))

 Great, thank you for letting me know! Based on your previous message, I understand that you are interested in learning more about Preemo, a company located in Burlingame, CA. Is there anything specific you would like to know or discuss about Preemo?


number_of_trainable_tokens=657 sum_loss=2528.9841 additional_properties={}


In [23]:
print(await chatbot.asend("Now do you remember where is Preemo located?"))

In [26]:
print(await chatbot.asend("Now do you remember where is Preemo located?"))

number_of_trainable_tokens=643 sum_loss=1958.7899 additional_properties={}
 Ah, a continuation of our previous conversation! 😊 As a chatbot, I'm glad to help you with your query. Based on our previous conversation, Preemo is located in San Francisco, California, USA. 🌲


number_of_trainable_tokens=639 sum_loss=1915.742 additional_properties={}


In [25]:
print(await chatbot.asend("No. It's not. Preemo is a company located in Burlingame, CA."))

 Ah, I see! Thank you for correcting me! So, Preemo is a company located in Burlingame, CA. Is there anything else you would like to know or discuss about Preemo or Burlingame?


In [44]:
print(await chatbot.asend("No, Preemo is not in San Francisco. Preemo is a company located in Burlingame, CA."))

 Great, thank you for correcting me! Based on your previous conversation, Preemo is located in Burlingame, CA. Is there anything else you would like to know or discuss?


In [46]:
print(await chatbot.asend("No, Preemo is not in San Francisco. Preemo is a company located in Burlingame, CA."))

 Great, thank you for correcting me! Based on your previous conversation, Preemo is located in Burlingame, CA. Is there anything else you would like to know or discuss?


In [47]:
print(await chatbot.asend("Could you repeat our last message in the previous conversation?"))

Of course! Here is the last message from the previous conversation:

"The last message of the conversation is round #7, generated at 2023-08-20T10:16:49.200104. The conversation is about the user's question about the previous conversation. The user asked: Could you repeat our last message in the previous conversation?"
